#### Notebook for running React experiments

In [1]:
import sys, os
import logging
sys.path.append('..')
root  = '../root/'

In [2]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
import json
import joblib
from util import summarize_react_trial, log_react_trial, save_agents, save_results
from agents import ReactReflectAgent, ReactAgent, ReflexionStrategy
from llm import AnyOpenAILLM

#### Load the HotpotQA Sample

In [4]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

#### Define the Reflexion Strategy

In [5]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [6]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Define the LLMs

In [7]:
action_llm = "gpt-3.5-turbo"
self_reflect_llm = "gpt-3.5-turbo"

#### Initialize a React Agent for each question

In [8]:
agent_cls = ReactReflectAgent if strategy != ReflexionStrategy.NONE else ReactAgent
if strategy != ReflexionStrategy.NONE:
    agents = [ReactReflectAgent(
                    row['question'],
                    row['answer'],
                 react_llm = AnyOpenAILLM(
                                             temperature=0,
                                             max_tokens=100,
                                             model_name=action_llm,
                                             model_kwargs={"stop": "\n"},
                                             openai_api_key=os.environ['OPENAI_API_KEY']),
                 reflect_llm = AnyOpenAILLM(
                                               temperature=0,
                                               max_tokens=250,
                                               model_name=self_reflect_llm,
                                               openai_api_key=os.environ['OPENAI_API_KEY']),
    ) for _, row in hotpot.iterrows()]
else:
    agents = [ReactAgent(
                    row['question'],
                    row['answer'],
                 react_llm = AnyOpenAILLM(
                                             temperature=0,
                                             max_tokens=100,
                                             model_name=action_llm,
                                             model_kwargs={"stop": "\n"},
                                             openai_api_key=os.environ['OPENAI_API_KEY']),
    ) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [9]:
n = 5
trial = 0
log = ''
results = []

run_dir = os.path.join(root, 'ReAct', strategy.value + '_'+ self_reflect_llm)
os.makedirs(run_dir, exist_ok=True)

# Logging config that writes to a file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)-8s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler(os.path.join(run_dir, 'log.txt')),
    ]
)

In [11]:
for i in range(1, n):
    for agent in [a for a in agents if not a.is_correct()]:
        if strategy != ReflexionStrategy.NONE:
            agent.run(reflect_strategy = strategy)
        else:
            agent.run()
        logging.info(f'Answer: {agent.key}')
    trial += 1
    log += log_react_trial(agents, trial)
    correct, incorrect, halted = summarize_react_trial(agents)
    results.append({"trial": trial, "correct": len(correct), "incorrect": len(incorrect), "halted": len(halted)})
    save_results(agents, results, run_dir)
    #save_agents(agents, os.path.join(run_dir, 'ReAct', strategy.value, 'agents', f'trial_{trial}'))
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}, Halted: {len(halted)}')

You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
Reflections:
- I was unable to find the new name or acronym for VIVA Media AG. I should have tried searching for their name change in 2004 and see if it mentions the new acronym. Additionally, I could have tried searching for similar companies or media organizations that underwent a name change in 2004 to see if any of them had a similar acronym.
You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
Reflections:
- I failed to find the answer because I did not have a specific name or information to search for the first governor a

/Users/becklabash/Documents/Research/reflexion/reflexion/env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/becklabash/Documents/Research/reflexion/reflexion/env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
Reflections:
- The possible reason for failure in this trial is the lack of specific keywords used in the search. The agent should have used more specific keywords related to the assassination and the Rome Protocols. To mitigate this failure, the agent should devise a new plan that includes searching for keywords such as "assassination Rome Protocols" or "assassination of Prime Ministers involved in Rome Protocols" to find more relevant information about the assassination. Additionally, the agent should consider searching for alternative events or contexts that may be related to the question in case the specific information about the assassination cannot be found.
You have attempted to answer following question before and failed. 

RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-QDvHd5gqx8LtTZII3OVsnyDz on tokens per min. Limit: 90000 / min. Current: 88933 / min. Contact us through our help center at help.openai.com if you continue to have issues.

#### Save the result log

In [ ]:
with open(os.path.join(run_dir, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join('ReAct', strategy.value, 'agents'))